In [63]:
import pandas as pd
import numpy as np
from scipy import stats, special
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (accuracy_score, roc_auc_score, confusion_matrix)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pyreadr

In [6]:
biomarker_clean = pyreadr.read_r("../data/biomarker-clean.RData")['biomarker_clean']
biomarker_clean.head()

,group,ados,CHIP,CEBPB,NSE,PIAS4,IL-10 Ra,STAT3,IRF1,c-Jun,...,UB2G2,Transgelin-2,ATPO,Corticotropin-lipotropin,QORL1,PEDF,CATF,FTCD,UBP25,PLXB2
0,ASD,8.0,0.335009,0.520303,-0.554298,0.649609,-0.357510,0.305328,-0.484193,0.308533,...,-0.283945,-0.258694,0.458736,1.133789,0.858608,-0.331575,-1.795728,3.000000,-0.477235,-1.234194
1,ASD,21.0,-0.071454,1.006274,3.000000,1.278818,-0.132677,1.133698,0.253024,0.407903,...,-0.108301,0.142860,-0.022246,-0.612397,-1.184603,-0.467821,-1.286492,0.783710,0.138254,0.095212
2,ASD,12.0,-0.406015,-0.531037,-0.059221,1.129386,0.553756,-0.333915,0.286523,-0.844532,...,-0.737726,-0.474468,-0.581352,-0.692143,0.272451,-1.340909,1.593076,-1.145940,-0.689551,0.838586
3,ASD,20.0,-0.101941,-0.250912,1.473261,0.077316,-0.704625,0.892828,2.607385,-0.372294,...,-0.932992,-0.140282,-0.740797,2.135767,-0.277749,0.149180,0.039046,-0.096161,-0.353567,-0.900581
4,ASD,22.0,-0.395238,-0.536000,0.041022,-0.298900,-0.830069,0.898742,1.014317,-0.843280,...,-0.728662,0.124020,-0.662626,-0.210703,-0.780781,0.187370,-0.407526,0.237886,-0.522804,0.504190


## Multiple Testing

In [17]:
def test_fn(df, alpha=0.05):
    group1, group2 = 'ASD', 'TD'
    data1 = df.loc[df['group'] == group1, 'level'].dropna()
    data2 = df.loc[df['group'] == group2, 'level'].dropna()

    n1, n2 = len(data1), len(data2)
    mean1, mean2 = np.mean(data1), np.mean(data2)
    var1, var2 = np.var(data1, ddof=1), np.var(data2, ddof=1)

    # Welch t-test
    t_stat, p_val = stats.ttest_ind(data1, data2, equal_var=False, alternative='two-sided')

    # Welch-Satterthwaite df
    df_num = (var1/n1 + var2/n2)**2
    df_den = (var1**2 / ((n1**2)*(n1-1))) + (var2**2 / ((n2**2)*(n2-1)))
    df_ = df_num / df_den if df_den != 0 else np.nan

    # Mean difference (estimate)
    estimate = mean1 - mean2

    # Standard error for CI
    se = np.sqrt(var1/n1 + var2/n2)

    # t critical for 95% CI
    t_crit = stats.t.ppf(1 - alpha/2, df_)
    lower_ci = estimate - t_crit * se
    upper_ci = estimate + t_crit * se

    return pd.Series({
        'group1': group1,
        'group2': group2,
        'n1': n1,
        'n2': n2,
        'mean1': mean1,
        'mean2': mean2,
        'statistic': t_stat,
        't_df': df_,
        'p_value': p_val,
        'alternative': 'two.sided',
        'estimate': estimate,
        'lower_ci': lower_ci,
        'upper_ci': upper_ci
    })

In [27]:
ttests_temp = biomarker_clean\
    .drop(columns = ['ados'], errors = 'ignore')\
        .melt(id_vars='group', var_name='protein', value_name='level')

ttests_out = (
        ttests_temp\
        .groupby('protein', group_keys=False)\
        .apply(test_fn)\
        .reset_index()\
    ).sort_values('p_value')\
        .reset_index(drop=True)


m = len(ttests_out)
hm = np.log(m) + 1/(2*m) - special.digamma(1)
ttests_out['m'] = m
ttests_out['hm'] = hm
ttests_out['rank'] = np.arange(1, m + 1)
ttests_out['p.adj'] = m * hm * ttests_out['p_value'] / ttests_out['rank']

/var/folders/29/4v1zzs5168q28y7f3hr88wnc0000gn/T/ipykernel_10049/2847007112.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(test_fn)\


In [28]:
ttests_out.head()

,protein,group1,group2,n1,n2,mean1,mean2,statistic,t_df,p_value,alternative,estimate,lower_ci,upper_ci,m,hm,rank,p.adj
0,DERM,ASD,TD,76,78,-0.448231,0.436606,-6.104833,151.410296,8.268940e-09,two.sided,-0.884838,-1.171205,-0.598471,1317,7.760707,1,0.000085
1,RELT,ASD,TD,76,78,-0.408347,0.366627,-5.647419,151.537306,7.818446e-08,two.sided,-0.774974,-1.046098,-0.503850,1317,7.760707,2,0.000400
2,FSTL1,ASD,TD,76,78,-0.396739,0.386566,-5.267203,151.850259,4.663516e-07,two.sided,-0.783305,-1.077120,-0.489490,1317,7.760707,3,0.001589
3,C1QR1,ASD,TD,76,78,-0.396144,0.385987,-5.261301,151.948374,4.788545e-07,two.sided,-0.782131,-1.075834,-0.488429,1317,7.760707,4,0.001224
4,Calcineurin,ASD,TD,76,78,-0.385474,0.348845,-5.238443,150.553143,5.371444e-07,two.sided,-0.734319,-1.011291,-0.457347,1317,7.760707,5,0.001098


## Random Forest

In [31]:
proteins_s1 = ttests_out.nsmallest(10, 'p.adj')['protein'].tolist()
proteins_s1

['DERM',
 'RELT',
 'Calcineurin',
 'C1QR1',
 'MRC2',
 'IgD',
 'CXCL16, soluble',
 'PTN',
 'FSTL1',
 'Cadherin-5']

In [33]:
predictors = biomarker_clean.drop(columns=['ados', 'group'], errors='ignore')
predictors.head()

,CHIP,CEBPB,NSE,PIAS4,IL-10 Ra,STAT3,IRF1,c-Jun,Mcl-1,OAS1,...,UB2G2,Transgelin-2,ATPO,Corticotropin-lipotropin,QORL1,PEDF,CATF,FTCD,UBP25,PLXB2
0,0.335009,0.520303,-0.554298,0.649609,-0.357510,0.305328,-0.484193,0.308533,1.566621,-0.579327,...,-0.283945,-0.258694,0.458736,1.133789,0.858608,-0.331575,-1.795728,3.000000,-0.477235,-1.234194
1,-0.071454,1.006274,3.000000,1.278818,-0.132677,1.133698,0.253024,0.407903,0.064303,0.133919,...,-0.108301,0.142860,-0.022246,-0.612397,-1.184603,-0.467821,-1.286492,0.783710,0.138254,0.095212
2,-0.406015,-0.531037,-0.059221,1.129386,0.553756,-0.333915,0.286523,-0.844532,1.415334,-0.548521,...,-0.737726,-0.474468,-0.581352,-0.692143,0.272451,-1.340909,1.593076,-1.145940,-0.689551,0.838586
3,-0.101941,-0.250912,1.473261,0.077316,-0.704625,0.892828,2.607385,-0.372294,-0.466765,-0.413388,...,-0.932992,-0.140282,-0.740797,2.135767,-0.277749,0.149180,0.039046,-0.096161,-0.353567,-0.900581
4,-0.395238,-0.536000,0.041022,-0.298900,-0.830069,0.898742,1.014317,-0.843280,-1.150820,0.300182,...,-0.728662,0.124020,-0.662626,-0.210703,-0.780781,0.187370,-0.407526,0.237886,-0.522804,0.504190


In [39]:
resp = biomarker_clean['group']
label_encoder = LabelEncoder()
response = label_encoder.fit_transform(resp)

In [47]:
rf = RandomForestClassifier(n_estimators=1000, random_state=101422, n_jobs=-1, oob_score=True)
rf.fit(predictors, response)

,n_estimators,1000
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,True


In [48]:
response_pred = rf.predict(predictors)
cm = confusion_matrix(response, response_pred)
cm

array([[76,  0],
       [ 0, 78]])

In [52]:
importance_df = pd.DataFrame({
    'protein': predictors.columns,
    'MeanDecreaseGini': rf.feature_importances_
}).sort_values('MeanDecreaseGini', ascending=False)

proteins_s2 = importance_df.head(10)['protein'].tolist()
proteins_s2

['DERM',
 'IgD',
 'MAPK14',
 'eIF-4H',
 'Notch 1',
 'ALCAM',
 'FSTL1',
 'PTN',
 'RET',
 'SOST']

## Logistic Regression

In [54]:
proteins_sstar = list(set(proteins_s1) & set(proteins_s2))
proteins_sstar

['PTN', 'DERM', 'FSTL1', 'IgD']

In [57]:
biomarker_sstar = biomarker_clean[['group'] + proteins_sstar].copy()
biomarker_sstar['class'] = biomarker_sstar['group'] == 'ASD'
biomarker_sstar = biomarker_sstar.drop(columns=['group'])
biomarker_sstar.head()
biomarker_sstar.tail()

,PTN,DERM,FSTL1,IgD,class
149,1.536982,1.542677,0.912191,0.198057,False
150,1.641185,0.653368,1.288914,0.907693,False
151,1.347364,0.745893,0.929612,-0.226453,False
152,-0.121617,0.228634,0.432834,0.278487,False
153,3.000000,1.332759,2.346258,1.056374,False


In [59]:
X = biomarker_sstar.drop(columns=['class'])
y = biomarker_sstar['class'].astype(int)  # 1 = ASD, 0 = TD

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101422, stratify=y)

In [61]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [62]:
y_prob_log = logreg.predict_proba(X_test)[:, 1]
y_pred_log = (y_prob_log > 0.5).astype(int)

In [65]:
cm_log = confusion_matrix(y_test, y_pred_log)
tn, fp, fn, tp = cm_log.ravel()

sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
accuracy = accuracy_score(y_test, y_pred_log)
roc_auc = roc_auc_score(y_test, y_prob_log)

In [66]:
class_metrics = pd.DataFrame({
    'metric': ['sensitivity', 'specificity', 'accuracy', 'roc_auc'],
    'value': [sensitivity, specificity, accuracy, roc_auc]
})

print(class_metrics)

        metric     value
0  sensitivity  0.800000
1  specificity  0.875000
2     accuracy  0.838710
3      roc_auc  0.804167
